In [21]:
!pip install tflite_model_maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [5]:
# Choose an object detection model archiecture
spec = model_spec.get('efficientdet_lite0')

In [25]:
!wget https://transfer.sh/pSAxRb/cards_dataset.zip

--2023-01-03 21:33:38--  https://transfer.sh/pSAxRb/cards_dataset.zip
Resolving transfer.sh (transfer.sh)... 144.76.136.153, 2a01:4f8:200:1097::2
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4988283 (4.8M) [application/zip]
Saving to: ‘cards_dataset.zip’

cards_dataset.zip   100%[===================>]   4.76M  5.70MB/s    in 0.8s    

2023-01-03 21:33:40 (5.70 MB/s) - ‘cards_dataset.zip’ saved [4988283/4988283]



In [ ]:
!unzip /content/cards_dataset.zip

In [27]:
import os
cwd = os.getcwd()

In [28]:
# Load the dataset
CLASSES = ['card']
TRAIN_DATASET_PATH = f'{cwd}/cards_dataset/train'
VALID_DATASET_PATH = f'{cwd}/cards_dataset/valid'
TEST_DATASET_PATH = f'{cwd}/cards_dataset/test'

train_data = object_detector.DataLoader.from_pascal_voc(
    TRAIN_DATASET_PATH,
    TRAIN_DATASET_PATH,
    CLASSES
)

val_data = object_detector.DataLoader.from_pascal_voc(
    VALID_DATASET_PATH,
    VALID_DATASET_PATH,
    CLASSES
)

test_data = object_detector.DataLoader.from_pascal_voc(
    TEST_DATASET_PATH,
    TEST_DATASET_PATH,
    CLASSES
)


In [29]:
# Train the TensorFlow model with the training data
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=val_data)

Epoch 1/50
 6/16 [==========>...................] - ETA: 2s - det_loss: 1.7964 - cls_loss: 1.1446 - box_loss: 0.0130 - reg_l2_loss: 0.0630 - loss: 1.8594 - learning_rate: 0.0083 - gradient_norm: 1.2540

16/16 [==============================] - 48s 502ms/step - det_loss: 1.7239 - cls_loss: 1.1122 - box_loss: 0.0122 - reg_l2_loss: 0.0630 - loss: 1.7869 - learning_rate: 0.0090 - gradient_norm: 1.2756 - val_det_loss: 1.5444 - val_cls_loss: 1.0112 - val_box_loss: 0.0107 - val_reg_l2_loss: 0.0630 - val_loss: 1.6074
Epoch 2/50
16/16 [==============================] - 6s 373ms/step - det_loss: 1.3429 - cls_loss: 0.8578 - box_loss: 0.0097 - reg_l2_loss: 0.0630 - loss: 1.4059 - learning_rate: 0.0100 - gradient_norm: 2.2265 - val_det_loss: 1.8972 - val_cls_loss: 1.4813 - val_box_loss: 0.0083 - val_reg_l2_loss: 0.0630 - val_loss: 1.9602
Epoch 3/50
16/16 [==============================] - 5s 327ms/step - det_loss: 0.8972 - cls_loss: 0.5047 - box_loss: 0.0079 - reg_l2_loss: 0.0630 - loss: 0.9603 - learning_rate: 0.0099 - gradient_norm: 2.2563 - val_det_loss: 0.9011 - val_cls_loss: 0.5726 - val_box_loss: 0.0066 - val_reg_l2_loss: 0.0630 - val_loss: 0.9641
Epoch 4/50
16/16 [==========================

In [30]:
# Evaluate the model with the test data
model.evaluate(test_data)

1/1 [==============================] - 4s 4s/step



{'AP': 0.8202348,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.8203406,
 'ARmax1': 0.33846155,
 'ARmax10': 0.84615386,
 'ARmax100': 0.84615386,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.84615386,
 'AP_/card': 0.8202348}

In [32]:
# Export as a TensorFlow Lite model
model.export(export_dir='.')

In [35]:
model.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    multiple                  3234464   
                                                                 
 class_net/class-predict (Se  multiple                 1746      
 parableConv2D)                                                  
                                                                 
 box_net/box-predict (Separa  multiple                 2916      
 bleConv2D)                                                      
                                                                 
Total params: 3,239,126
Trainable params: 3,191,990
Non-trainable params: 47,136
_________________________________________________________________


In [34]:
!pwd

/content
